# Langchain crash course

In [1]:
import os
from pprint import pprint
import json

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Spice Affair"


In [4]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n"Spice Palace"'

## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

{
    "cuisine": "Mexican",
    "text": "\n\n\"La Cantina de Sabores\""
}


In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
{
    "cuisine": "Mexican",
    "text": "\n\n\"La Cantina de Sabores\" (The Cantina of Flavors)"
}


In [8]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [9]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

results = chain.invoke("Indian")
print(results["input"])
pprint(results["output"])

Indian
('\n'
 '\n'
 '1. Butter Chicken: tender chicken cooked in a rich and creamy tomato-based '
 'sauce\n'
 '2. Tandoori Lamb Chops: marinated lamb chops grilled in a clay oven\n'
 '3. Vegetable Biryani: fragrant basmati rice cooked with mixed vegetables and '
 'aromatic spices\n'
 '4. Palak Paneer: homemade cottage cheese cooked in a creamy spinach gravy\n'
 '5. Chicken Tikka Masala: grilled chicken cooked in a spicy and tangy '
 'tomato-based sauce\n'
 '6. Naan Bread: traditional Indian flatbread baked in a clay oven\n'
 '7. Samosas: crispy pastry filled with spiced potatoes and peas\n'
 '8. Malai Kofta: vegetable and cheese dumplings served in a creamy cashew '
 'sauce\n'
 '9. Mango Lassi: refreshing yogurt-based drink with fresh mango puree\n'
 '10. Gulab Jamun: deep-fried milk dumplings soaked in a sweet syrup for '
 'dessert.')


#### Sequential Chain

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
results = chain.invoke({"cuisine": "Indian"})
pprint(results["cuisine"])
pprint(results["restaurant_name"])
pprint(results["menu_items"])

'Indian'
'\n"Maharaja\'s Delight" '
('\n'
 '\n'
 '1. Butter Chicken: Tender chicken cooked in a creamy tomato and butter '
 'sauce.\n'
 '2. Tandoori Lamb Chops: Marinated lamb chops cooked in a clay oven.\n'
 '3. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables and '
 'Indian spices.\n'
 '4. Dal Makhani: Creamy black lentils slow-cooked with butter and spices.\n'
 '5. Aloo Gobi: Cauliflower and potatoes cooked with onions, tomatoes, and '
 'spices.\n'
 '6. Palak Paneer: Homemade cottage cheese cooked in a spinach and cream '
 'sauce.\n'
 '7. Chicken Tikka Masala: Grilled chicken cooked in a rich tomato and onion '
 'sauce.\n'
 '8. Naan Bread: Traditional Indian flatbread, served hot and fresh.\n'
 '9. Mango Lassi: A refreshing yogurt drink made with mango puree and spices.\n'
 '10. Gulab Jamun: Deep-fried milk dumplings soaked in a sweet syrup, served '
 'as a dessert.')


## Agents

In [14]:
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [15]:
from langchain import hub
from langchain.agents import AgentType, create_react_agent, load_tools, AgentExecutor

In [16]:
prompt = hub.pull("hwchase17/react")

In [20]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What was the GDP of US in 2022?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should search for the answer using a search engine.
Action: Search
Action Input: "GDP of US in 2022"25439.70 USD Billion I should use a calculator to convert the result to a more readable format.
Action: Calculator
Action Input: 25439.70 * 1000000000Answer: 25439700000000.0 I now know the final answer.
Final Answer: The GDP of US in 2022 is 25.44 trillion USD.

> Finished chain.


{'input': 'What was the GDP of US in 2022?',
 'output': 'The GDP of US in 2022 is 25.44 trillion USD.'}

#### Wikipedia and llm-math tool

In [21]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "When was Elon musk born? What is the square root of his age as of 2023?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should use Wikipedia to find out when Elon Musk was born and then use a calculator to find the square root of his age.
Action: Wikipedia
Action Input: Elon MuskPage: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acq

{'input': 'When was Elon musk born? What is the square root of his age as of 2023?',
 'output': '7.211102550927978'}

In [22]:
7.211102550927978 ** 2

51.99999999999999

## Memory

#### ConversationBufferMemory

In [29]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabroso Cantina"'}


In [32]:
name = chain.invoke("Arabic")
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Mexican\nAI: \n\n"El Sabroso Cantina"\nHuman: Arabic\nAI: \n\n"Al-Fawarish" (meaning "The Treasures" in Arabic)', 'text': '\n\n"Al-Fawarish" (meaning "The Treasures" in Arabic)'}


In [33]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabroso Cantina"
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [34]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [36]:
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia by 17 runs in the final match held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Michael Holding. The tournament featured eight teams and was organized by the International Cricket Council (ICC). The West Indies team dominated the tournament, winning all their matches and ultimately becoming the first ever cricket world cup champions.",
 'response': ' As I mentioned before, the West Indies team won the first cricket world cup in 1975. Is there any other information you would like to know about the tournament or the winning team?'}

In [37]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia by 17 runs in the final match held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Michael Holding. The tournament featured eight teams and was organized by the International Cricket Council (ICC). The West Indies team dominated the tournament, winning all their matches and ultimately becoming the first ever cricket world cup champions.\nHuman: Who won the first cricket world cup?\nAI:  As I mentioned before, the West Indies team won the first cricket world cup in 1975. Is there any other information you would like to know about the tournament or the winning team?",
 'response': '  5+5 equals 10.'}

In [38]:
convo.invoke("Who was the captain ofthe winning team?")

{'input': 'Who was the captain ofthe winning team?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia by 17 runs in the final match held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Michael Holding. The tournament featured eight teams and was organized by the International Cricket Council (ICC). The West Indies team dominated the tournament, winning all their matches and ultimately becoming the first ever cricket world cup champions.\nHuman: Who won the first cricket world cup?\nAI:  As I mentioned before, the West Indies team won the first cricket world cup in 1975. Is there any other information you would like to know about the tournament or the winning team?\nHuman: How much is 5+5?\nAI:   5+5 equals 10.",
 'response': ' The captain of the West Indies team during the 1975 cricket world cup was Cliv

In [39]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia by 17 runs in the final match held at Lord's Cricket Ground in London, England. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Michael Holding. The tournament featured eight teams and was organized by the International Cricket Council (ICC). The West Indies team dominated the tournament, winning all their matches and ultimately becoming the first ever cricket world cup champions.
Human: Who won the first cricket world cup?
AI:  As I mentioned before, the West Indies team won the first cricket world cup in 1975. Is there any other information you would like to know about the tournament or the winning team?
Human: How much is 5+5?
AI:   5+5 equals 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team during the 1975 cricket world cup was Clive Lloyd. He was a well-respected lea

#### ConversationBufferWindowMemory

In [41]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was held in 1975 and was won by the West Indies. They defeated Australia by 17 runs in the final match at Lord's Cricket Ground in London. The West Indies team was led by Clive Lloyd and included notable players such as Viv Richards, Gordon Greenidge, and Michael Holding."}

In [42]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies. They defeated Australia by 17 runs in the final match at Lord's Cricket Ground in London. The West Indies team was led by Clive Lloyd and included notable players such as Viv Richards, Gordon Greenidge, and Michael Holding.",
 'response': ' The answer to 5+5 is 10. This is a basic arithmetic equation that can be solved by adding the two numbers together. In mathematics, this is known as an addition operation, and the result is called the sum. Did you know that the concept of addition has been around since ancient civilizations, such as the Egyptians and Babylonians? They used different methods, such as counting on fingers or using tally marks, to solve addition problems.'}

In [43]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: How much is 5+5?\nAI:  The answer to 5+5 is 10. This is a basic arithmetic equation that can be solved by adding the two numbers together. In mathematics, this is known as an addition operation, and the result is called the sum. Did you know that the concept of addition has been around since ancient civilizations, such as the Egyptians and Babylonians? They used different methods, such as counting on fingers or using tally marks, to solve addition problems.',
 'response': ' I cannot determine the captain of the winning team without more context. Can you provide me with the name of the team or the sport in question?'}